# You may only delete 12 items if betfair is listed, if not then don't 
# 
# You should also make sure that the scraper only selects the pages where it finds PinnacleODDS--> FIND UNIQUE IDENTIFIER!!

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime #as dt
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
import sys
import warnings
from datetime import timedelta
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import time 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

In [333]:
def load_exists(driver):
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.Button.SportsBoxAll.LoadMore")))
        return True
    except:
        return False

def cookies(driver):
    try:
        driver.find_element_by_xpath('//*[@id="app"]/div/span[2]/div/button')
        return True
    except:
        return False       
        
def click_through(x):
    if load_exists(x):
        if cookies(x):
            pisso= x.find_element_by_xpath('//*[@id="app"]/div/span[2]/div/button')
            ActionChains(x).move_to_element(pisso).click().perform()
            element= x.find_element_by_css_selector("button.Button.SportsBoxAll.LoadMore")
            ActionChains(x).move_to_element(element).click().perform()
            click_through(x)
        else:
            element= x.find_element_by_css_selector("button.Button.SportsBoxAll.LoadMore")
            ActionChains(x).move_to_element(element).click().perform()
            click_through(x)
    return x

def get_links(browse):
    rows= []
    for thing in BeautifulSoup(browse.page_source,'lxml').find_all('li'):
        if thing.find('a',{'class':'MatchTitleLink'}) and \
        pd.to_datetime(datetime.now()) <= pd.to_datetime(thing.find('span', {'class':'DateTime'}).text): 
            dict1= {}
            match_link= 'https://www.betbrain.de' + thing.find('a', {'class':'MatchTitleLink'})['href']
            date= pd.to_datetime(thing.find('span', {'class':'DateTime'}).text)
            dict1.update({'match_link':match_link,'date':date})
            rows.append(dict1)
    df= pd.DataFrame(rows)
    df=df[df['match_link'].str.contains("home-draw-away")& df['match_link'].str.contains("football")].reset_index(drop= True)
    return df

chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--headless')
browser = webdriver.Chrome(
executable_path=chrome_path, options=chrome_options)
browser.get('https://www.betbrain.de/next-matches/')

doc= click_through(browser)

links= get_links(doc)

TypeError: 'Timestamp' object is not iterable

In [416]:
def xpath_exists(text,driver):
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,text)))
        return True
    except:
        return False

def get_odds(URL,playtime):
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    browser.get(URL)
    if xpath_exists("//*[contains(text(), 'Pinnacle Sports')]",browser):
        if xpath_exists('//*[@id="app"]/div/span/div/button',browser):
            element= browser.find_element_by_xpath('//*[@id="app"]/div/span/div/button')
            ActionChains(browser).move_to_element(element).click().perform()
        
        country= browser.find_element_by_xpath('/html/body/div[1]/div/section/div/div/div/div[2]/div/div[1]/ul/li[4]/a/span[1]').text
        league= browser.find_element_by_xpath('/html/body/div[1]/div/section/div/div/div/div[2]/div/div[1]/ul/li[5]/a/span[1]').text
        html_page= browser.page_source
        browser.quit()
        soup= BeautifulSoup(html_page)
        date= playtime 
        home_team= soup.find('p',{'class':'ScoresHome'}).findAll('span')[1].text
        away_team= soup.find('p',{'class':'ScoresAway'}).findAll('span')[1].text
        scrape_time= pd.to_datetime(datetime.now())
        bookie_list= []
        for thing in soup.findAll('div', {'class':'OTBookmakers'}):#
            for element in thing.findAll('span', {'class':'BookieLogo BL'}):
                bookie_list.append(element.find('span').text)
        outcome= ['H','D','A']   
        exclude= ['Betfair Exchange','Betfair']
        bookie_cols= [x.replace(' ','')+y for x in bookie_list for y in outcome if x not in exclude]
        odds= []
        for element in soup.findAll('a',{'class':['OTOddsLink', 'HasDeeplink']}):
            odds.append(element.next_element.text[:-2])
        keys= ['home_team','away_team','date','scrape_time','country','league']+bookie_cols
        values= [home_team, away_team, date,scrape_time,country,league]+odds[:-12]
        zipper= zip(keys, values)
        match_dict= dict(zipper)
    else:
        match_dict= {}
        
    return match_dict

In [433]:
start_time = time.time()

estelle= []
for i in range(0, len(links)):
    estelle.append(get_odds(links['match_link'][i], links['date'][i]))
    
print (time.time() - start_time, "seconds")
match_frame= pd.DataFrame(estelle)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/section/div/div/div/div[2]/div/div[1]/ul/li[4]/a/span[1]"}
  (Session info: headless chrome=90.0.4430.212)


In [436]:
#match_frame
#131/7*109/3600
pd.DataFrame(estelle)

,home_team,away_team,date,scrape_time,country,league,Betano.deH,Betano.deD,Betano.deA,22BetH,...,BetagoA,Bet365H,Bet365D,Bet365A,BetClic.frH,BetClic.frD,BetClic.frA,Fortuna.roH,Fortuna.roD,Fortuna.roA
0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mashujaa FC,Pan African FC,2021-05-20 13:00:00,2021-05-20 14:11:01.053153,Tansania,Second Division 2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mbale Heroes,Paidha Black Angels,2021-05-20 13:00:00,2021-05-20 14:11:18.050707,Uganda,2nd Division 2021,1.44,4.05,6.40,1.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kirsehir Belediye Spor,Sakaryaspor,2021-05-20 13:00:00,2021-05-20 14:11:35.337688,Türkei,2. Lig 2020/2021,2.57,2.75,3.00,2.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vital'O FC,AS Inter Star,2021-05-20 13:00:00,2021-05-20 14:11:52.211735,Burundi,Premier League 2020/2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AS Cheminots de Pointe-Noire,Racing Club Brazzaville,2021-05-20 13:00:00,2021-05-20 14:12:17.465954,Kongo,Ligue 1 2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Terrazzo and Tiles,Ndejje University,2021-05-20 13:00:00,2021-05-20 14:12:34.390761,Uganda,2nd Division 2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tooro United FC,Proline FC,2021-05-20 13:00:00,2021-05-20 14:12:50.986225,Uganda,2nd Division 2021,1.95,3.20,3.60,1.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Kigezi Homeboys,Water FC,2021-05-20 13:00:00,2021-05-20 14:13:07.796451,Uganda,2nd Division 2021,2.55,3.15,2.57,2.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ihefu FC,Dodoma,2021-05-20 13:00:00,2021-05-20 14:13:24.837553,Tansania,Premier League 2020/2021,2.20,3.00,3.20,2.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [415]:
links['match_link'][37]


NameError: name 'html' is not defined